このノートブックではpythonによるtitanicデータ処理の基本を勉強していきます。<br>
ではまずpandasというデータ処理用のライブラリを用いてデータを読み込んでみましょう。<br>
pythonではライブラリを読み込むときにimportもしくはfromを用います。<br>
ではpandasを読み込んでみましょう。<br>
このノートブックではtitanic_datasフォルダにデータがあると仮定して進めます。kaggleのkernelを使用している場合、データは../input/にあるのでデータのパスをtitanic_darasから../inputに変更してください

In [ ]:
import pandas
read_data = pandas.read_csv("titanic_datas/train.csv")

無事にデータを読み込むことはできたでしょうか？できたか確認してみましょう。

In [ ]:
read_data

もしうまく読み込めていたら表のようなものが出力されていると思います。

titanicではどのような人が優先的に救助されたのでしょうか？<br>
おそらく、レディーファーストで女性で若い人が優先されて救助されたのは間違いないでしょう。では生存率を見てみましょう、実際にそうだったのでしょうか。

In [ ]:
male_data = read_data[read_data.Sex == "male"]
print(len(male_data[male_data.Survived == 1]) / len(male_data))

female_data = read_data[read_data.Sex == "female"]
print(len(female_data[female_data.Survived == 1]) / len(female_data))

これから女性の生存率がはるかに高いことがわかります。<br>
ではほかの要素はどうでしょうか？例えば年齢などは関係してくるのでしょうか？<br>
では試しにグラフを書いてみましょう。<br>

In [ ]:
import matplotlib.pyplot as plt

split_data = []
for survived in [0,1]:
    split_data.append(read_data[read_data.Survived==survived])

temp = [read_data.Age.dropna() for i in split_data]
plt.hist(temp, histtype="barstacked", bins=100)
plt.show()

グラフだと若いと生存率が高いように見えます。本当でしょうか、割合を見てみましょう。

In [ ]:
read_data.Age.fillna(read_data.Age.median(), inplace=True)
age_list = list(set(map(int, read_data.Age)))
age_list.sort()
age_per = list()
for i in age_list:
    age_data = read_data[read_data.Age == i]
    if len(age_data) != 0:
        age_per.append(len(age_data[age_data.Survived == 1]) / len(age_data))
        print(i, len(age_data[age_data.Survived == 1]) / len(age_data))
    else:
        age_per.append(0)

では、これを折れ線グラフにしてみましょう

In [ ]:
plt.plot(age_list, age_per)
plt.show()

このグラフから年齢が低い、もしくは高い人の生存率が高いことがわかります。<br>
ここまでで、性別および年齢と生存率の相関がわかりました。以下では性別、年齢、社会的階級が生存と関係していたとかていして話を進めます。
では各変数と生存率の相関を見るのはここまでにして、データの解析を行っていきましょう。

今回、このノートブックではランダムフォレストとサポートベクトルマシンの仕組みおよび実装の説明をしていきたいと思います。<br>
最初にランダムフォレストのもととなる決定木という手法について説明と実装をおこなっていきたいとおもいます。<br>
決定木はかんたんに言うと、データをもとにデータを分類するルールを作っていく手法です。<br>
ではどのようにして分類するルールを決定しているのでしょうか？それを今から見ていきましょう。

物事を決定するとき、決定しやすい情報がほしいと思います。<br>

例えば、政治家といわれるとその情報だけでどの政治家か断定するのは難しいでしょう。しかし、これが大統領だとどうでしょう、一気に候補に挙がる人が少なくなることがわかります。<br>
このように得た情報によって物事の決定のしやすさが大きく変わったと思います。<br>

このように人間のように物事を決定できたら楽なのですが、コンピュータは計算ぐらいしかできないのでどうにかしてこれらの「情報の大きさ」というものを数学的に表す必要があります。<br>
実はこのようなものを表すのに最適な数学の道具があります。それが情報量です。<br>

情報量Iはつぎのように定義される量です。
\begin{equation*}
I = -log P(x)
\end{equation*}
ここでP(x)はxが起きた時の確率を表しています。<br>
何故このような式になるのかは、深く考えないようにしてください。「あぁ、数学ではこのように情報の大きさを表しているんだな」というだけで十分です。

情報量はある物事が起きた時に得られる情報の大きさを表しています。しかし、ここでほしいのは「一回情報を得たときに得られる情報の期待値を最小にする物事」です。<br>
なぜこれを最小にしたいかというと、例えば先ほどの例を考えると政治家の母数よりも大統領の母数のほうが少ないですよね？このような情報では当然得られる情報量は少ないでしょう。つまり情報量が少なくなるように情報を選ぶと、少ない回数で物事を決定できることがわかります。<br>
では一回に得られる情報量の期待値とはどのように計算したら良いでしょうか。

ここで一度期待値を思い出してみましょう。<br>
期待値を覚えていますか？覚えていない方もいるかもしれませんが大丈夫です、これから覚えましょう。<br>
期待値Eとはある事象Xに対する確率P(X)とある事象Xに対応する関数の値f(X)に対して
\begin{equation*}
E= \sum_{X}^{}P(X)f(X)
\end{equation*}
で表せる量です。<br>
これは一回物事が起きた時に得られる平均のf(X)の値を表しています。<br>

つまりこれに先ほどの情報量の定義式を入れれば一回に得られる平均の情報量Hがわかることになります。したがって
\begin{equation*}
H = \sum_{X}^{} -P(X)log P(X)
\end{equation*}
これを平均情報量(エントロピー)といいます。

ではまずエントロピーを計算する関数を作ってみましょう。

In [ ]:
def entropy(probabilities):
    return sum(-p * math.log(p,2) for p in probabilities if p)

どうでしょうか？この関数はただ式をプログラムにするだけなので難しくなかったと思います。難しかったという方も大丈夫、僕もこれを書くとき悩みましたから。<br>
このプログラムでは確率を用います。なのでデータからあるデータが出てくる確率を計算する関数を作ってみましょう。<br>
ここで注意が必要で、確率の総和は1でなければなりません。ここだけは注意してください。

In [ ]:
def probabilities(labels):
    total_count = len(labels)
    return [count / total_count for count in Counter(labels).values()]   

これもそこまで難しくないと思います。ただし、Counterというクラスを使用しています。これは組み込み型でPythonで集計などをするとき便利な型なので覚えておくといいかもしれません。

ここまででエントロピーを計算する関数と確率を計算するプログラムができました。次にデータを受け取り、そのデータから各データのエントロピーを計算するプログラムを書いてみましょう。
ここではひとまずデータの構造はtuple型で最初にデータ、次に答えが来るようなtuple型のlistを仮定します。

    [
    (data, answer),
    (data, answer),
    ...
    ]

In [ ]:
def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    p = probabilities(labels)
    return entropy(p)

では次に分割されたデータのエントロピーを計算しましょう。

In [ ]:
def partition_entropy(subsets):
    total_count = sum(len(subset) for subset in subsets)
    return sum(data_entropy(subset) * len(subset) / total_count for subset in subsets)

では次にデータを与えられたキーで分割する関数を作ってみましょう。<br>
ここで与えられたキーに対応するデータを出力のdictのキーとします。つまり<br>

    [
        {1:2, 2:3}
        {1:3, 2:6}
        {1:2, 2:5}
    ]
    
というデータが与えられたとき、この関数にキーである1を与えると<br>

    {
    2:[{1:2, 2:3}, {1:2, 2:5}],
    4:[{1:3, 2:6}],
    }
    
などと帰ってくる関数です。

In [ ]:
def partition_by(inputs, attribute):
    groups = dict()
    for input in inputs:
        key = input[0][attribute]
        if not key in groups:
            groups[key] = list()
        groups[key].append(input)
    return groups

ここまでで、分割されたデータのエントロピーの計算、データの分割まで関数を作成しました。<br>
次に与えられたキーでデータを分割し、分割したデータのエントロピーを計算する関数を作ってみましょう。

In [ ]:
def calcurate_entropy(inputs, attribute):
    partitions = partition_by(inputs, attribute)
    #print(partitions.values())
    return partition_entropy(partitions.values())

ではこれらはきちんと動くのでしょうか？<br>
実際に年齢と性別のエントロピーを計算して確認してみましょう

In [ ]:
from collections import Counter, defaultdict
import math

data_key =["Age","Sex","Pclass"]

train = [({key:read_data.T[index][key] for key in data_key}, read_data.T[index]["Survived"]) for index in read_data.T]

for key in data_key:
    print(key, calcurate_entropy(train, key))

何らかの値が計算できたと思います。<br>
最初に書いてあるように、エントロピーが最小になるようにルールを決定したらよいので、この場合最初に性別で分けるのが良いことがわかります。

ここまでで、ルールを決定する方法を説明しました。<br>
しかしこれらのルールをいちいち計算して、人間の手で作成していくのは非効率的です。<br>
なのでルールを自動的に計算する関数を再帰を使って作りましょう。

手順としては、まずデータ数、Survivedが1であるデータ数、Survivedが0であるデータ数を計算します。<br>

Survivedが0,1であるデータ数のどちらかが0になったとき、データが決定されるので、ここでそのルールの生成を止めます。<br>
もしsplit_candidatesがFalseの場合、つまり分割の候補がない場合、より多いデータを返します。

それ以外の時、最小のエントロピーを持つキーでルールを生成したらよいので、最小のエントロピーを持つキーを計算します。<br>
そして、その最小のキーの中に存在するデータを分割し、分割したデータをもとに、そのデータからさらにルールの生成などを再帰的に行うことで自動生成ができます。<br>

In [ ]:
from functools import partial

def build_tree(inputs, split_candidates=None):
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()
    
    num_inputs = len(inputs)
    num_1 = len([label for item, label in inputs if label == 1])
    num_0 = num_inputs - num_1
    
    if num_1 == 0:
        return 0
    if num_0 == 0:
        return 1
    if not split_candidates:
        return 1 if num_1 >= num_0 else 0
    
    best_attribute = min(split_candidates, key=partial(calcurate_entropy, inputs))
    
    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates if a != best_attribute]
    
    sub_trees = {attribute_value : build_tree(subset, new_candidates) for attribute_value, subset in partitions.items()}
    sub_trees[None] = 1 if num_1 > num_0 else 0
    
    return best_attribute, sub_trees

では決定木を作ってみましょう。

In [ ]:
tree = build_tree(train)
tree

ではこれをもとにデータを分類してみたいと思います。
そのために分類木とデータからデータを分類する関数を作ってみましょう

In [ ]:
def classify(tree, input):
    if tree in [0, 1]:
        return tree
    
    attribute, subtree_dict = tree
    subtree_key = input.get(attribute)
    
    if not subtree_key in subtree_dict:
        subtree_key = None
    
    subtree = subtree_dict[subtree_key]
    return classify(subtree, input)

この関数にデータを入れたいと思います。

In [ ]:
read_test_data = pandas.read_csv("titanic_datas/test.csv")

input_data = [{key:read_data.T[index][key] for key in data_key} for index in read_data.T]
result = [classify(tree, input_data[i]) for i in range(len(read_test_data))]

read_test_data["Survived"] = result 
read_test_data[["PassengerId","Survived"]].to_csv("titanic_datas/submission.csv",index=False)

ここまでで、提出用のデータまで作成することができました。<br>
しかし、提出はちょっと待ってください。これを提出しても0.55くらいのスコアしか出ません。<br>
なぜかというと、この学習したデータのみに適応した決定木が作成されているためです。<br>
これに対してどのように対処したらよいでしょうか？

実はこれに対応するために作られたアルゴリズムがあります、それがランダムフォレストです。<br>
ではランダムフォレストはどのように実装したらよいでしょうか？